<a href="https://colab.research.google.com/github/samwang1228/Gemini-Chat-Room/blob/main/streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [2]:
%%writefile app.py

Writing app.py


# 獲取公網IP
! wget -q -O - ipv4.icanhazip.com

* wget 是一個從網絡上自動下載檔案的自由工具，支持HTTP、HTTPS和FTP協議。
-q 表示quiet模式，即不顯示任何下載資訊。
-O - 表示將下載的內容輸出到標準輸出（stdout）而非檔案。
ipv4.icanhazip.com 是一個提供服務的URL，當你訪問它時，它會返回你的公共IPv4地址。

In [3]:
! wget -q -O - ipv4.icanhazip.com

35.245.85.178


# 啟動服務
! streamlit run app.py & npx localtunnel --port 8501

這條指令實際上包含了兩個部分，通過&符號分隔，表示同時執行兩個命令：
* streamlit run app.py
  * streamlit 是一個用於快速創建和分享數據應用的Python庫。
  * run app.py 表示運行名為app.py的Streamlit應用程序。
* npx localtunnel --port 8501
  * npx 是一個Node.js的套件執行器，用於執行在package.json檔案中列出的套件。
  * localtunnel 是一個工具，可以將你本地的服務通過公共URL公開到互聯網上。
  * --port 8501 表示將本地的8501端口的服務公開出去。Streamlit默認在8501端口運行，因此這裡指的是將Streamlit應用公開到互聯網上。

In [9]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.245.85.178:8501

npx: installed 22 in 2.196s
your url is: https://cuddly-hotels-accept.loca.lt
  Stopping...
^C


# 主要code

In [ ]:
import streamlit as st
from PIL import Image
import google.generativeai as genai




with open("key.txt") as f:
  key=f.read()
genai.configure(api_key=key)
def get_gemini_response(model_name,prompt, temperature, top_k,top_p):
    generation_config = {
        "temperature": temperature,
        "top_p": 1,
        "top_k": top_k,
        "max_output_tokens": 2048,
    }

    model = genai.GenerativeModel(model_name
                                  ,generation_config=generation_config
                                  )

    response = model.generate_content(prompt)
    return response.text

# 初始化Streamlit應用
st.set_page_config(page_title="Gemini Image Chatbot")

# 定義各個頁面的功能
def home_page():
    st.header("Gemini應用")
    # 設置gemini 參數
    temperature = st.sidebar.slider('temperature', min_value=0.0, max_value=1.0, value=0.7)
    top_p = st.sidebar.slider('top_p', min_value=0.0, max_value=1.0, value=1.0)
    top_k = st.sidebar.number_input('Top K', min_value=1, max_value=10, value=1, step=1)
    # 設置聊天記錄
    if 'chat_history' not in st.session_state:
        st.session_state['chat_history'] = []

    # 聊天室佈局
    chat_container = st.container()

    # 創建form
    with st.form("chat_form", clear_on_submit=True):
        input_text = st.text_input("Your message:")
        uploaded_file = st.file_uploader("Upload an image (optional):", type=["jpg", "jpeg", "png"])
        submit_button = st.form_submit_button("Send")

        if uploaded_file is not None:
            # 顯示上傳完成提示
            st.success('Upload finished!')

        if submit_button and (input_text or uploaded_file):
            # submit且有內容
            if uploaded_file:
                # image and text
                image = Image.open(uploaded_file)
                response = get_gemini_response("gemini-pro-vision",[input_text, image],temperature,top_k,top_p)

            else:
                # only text
                response = get_gemini_response("gemini-1.0-pro",input_text,temperature,top_k,top_p)
                image = None

            # 新增至歷史紀錄
            st.session_state['chat_history'].append(("You", input_text, image))
            st.session_state['chat_history'].append(("Bot", response, None))

    with chat_container:
        for idx, (user, text, img) in enumerate(st.session_state['chat_history']):
            # 取得紀錄
            if user == "You":
                # 用戶的訊息和圖標
                st.markdown(f"<img src='https://resources.matcha-jp.com/resize/720x2000/2020/03/20-99819.webp' class='icon' style='width: 50px; height: auto;'> **You (message {idx}):**", unsafe_allow_html=True)
                st.markdown(f"{text}")
            else:
                # 機器人的訊息和圖標
                st.markdown(f"<img src='https://cdn-icons-png.flaticon.com/512/4712/4712035.png' class='icon' style='width: 50px; height: auto;'> **Bot replied (message {idx}):**", unsafe_allow_html=True)
                st.markdown(f"{text}")
            if img is not None:
                st.image(img, use_column_width=True)


    st.write("Welcome to the home page!")



def chatbot_page():
    st.title("Chatbot Page")
    st.write("Here you can interact with the chatbot.")

def settings_page():
    st.title("Settings Page")
    st.write("Adjust the application settings here.")

with st.sidebar:
    st.title("Navigation")
    page = st.radio("Choose a page:", ['Home', 'Chatbot', 'Settings'])

if page == 'Home':
    home_page()
elif page == 'Chatbot':
    chatbot_page()
elif page == 'Settings':
    settings_page()

